In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

## Join RAW files

In [3]:
DATA_PATH = os.path.join("..", "data")
pod_names = [
    "adservice", "cartservice", "checkoutservice", "currencyservice", "emailservice", "frontend", "paymentservice", "productcatalogservice", "recommendationservice", "redis", "shippingservice",
]

dataframes = []
for pod_name in tqdm(pod_names):
    cpu_file = os.path.join(DATA_PATH, "raw", f"{pod_name}_cpu.csv")
    dataframes.append(
        pd.read_csv(cpu_file, index_col="timestamp", names=["timestamp", f"{pod_name}_cpu"], header=0)
    )
    ram_file = os.path.join(DATA_PATH, "raw", f"{pod_name}_memory.csv")
    dataframes.append(
        pd.read_csv(ram_file, index_col="timestamp", names=["timestamp", f"{pod_name}_ram"], header=0)
    )
    
nodes_df = pd.concat(dataframes, axis=1, join="inner")

# nodes_df = (nodes_df - nodes_df.mean()) / nodes_df.std()
# nodes_df = (nodes_df - nodes_df.min()) / (nodes_df.max() - nodes_df.min())
nodes_df

100%|██████████| 11/11 [00:00<00:00, 17.52it/s]


,adservice_cpu,adservice_ram,cartservice_cpu,cartservice_ram,checkoutservice_cpu,checkoutservice_ram,currencyservice_cpu,currencyservice_ram,emailservice_cpu,emailservice_ram,...,paymentservice_cpu,paymentservice_ram,productcatalogservice_cpu,productcatalogservice_ram,recommendationservice_cpu,recommendationservice_ram,redis_cpu,redis_ram,shippingservice_cpu,shippingservice_ram
timestamp,,,,,,,,,,,,,,,,,,,,,
1701825744,0.006491,4.344186e+06,0.010313,2.792473e+03,0.005188,1.356858e+06,0.057166,4.505918e+06,0.004008,2.443887e+02,...,0.004668,2.030300e+06,0.021772,2.934991e+06,0.011848,5.497300e+06,0.004059,1.177182e+06,0.004627,1.403310e+03
1701825804,0.006662,1.122781e+03,0.006516,2.317005e+06,0.004203,1.181493e+02,0.047199,2.284438e+04,0.004733,0.000000e+00,...,0.003759,1.683843e+06,0.011461,1.618227e+06,0.011138,1.967024e+06,0.002752,1.420288e+06,0.003311,1.532223e+06
1701825864,0.005444,2.501221e+02,0.009085,1.911022e+06,0.003002,1.428579e+06,0.036961,1.115212e+04,0.004050,2.669992e+06,...,0.001572,4.198400e+03,0.017683,1.491514e+06,0.010565,1.563657e+02,0.003950,5.277394e+03,0.003696,1.363703e+06
1701825924,0.005505,0.000000e+00,0.009100,4.154800e+06,0.003821,1.409514e+06,0.053485,1.610893e+06,0.004510,4.283231e+06,...,0.004368,1.421130e+06,0.019390,1.280892e+06,0.009156,2.200314e+06,0.004141,1.435541e+06,0.003736,1.881729e+06
1701825984,0.002363,6.353886e+06,0.009596,4.152257e+06,0.003748,2.002629e+06,0.055062,1.341332e+04,0.004216,9.512971e+01,...,0.003510,1.601227e+06,0.018888,9.876798e+03,0.011243,4.731383e+06,0.003892,2.240093e+06,0.003977,3.218463e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701911904,0.005540,2.687047e+02,0.010791,2.166942e+06,0.003826,3.199345e+06,0.059259,1.982433e+06,0.004989,0.000000e+00,...,0.003935,1.157220e+04,0.015134,2.147919e+06,0.010035,4.496351e+06,0.004098,1.795620e+06,0.004481,2.855173e+06
1701911964,0.005088,3.671032e+06,0.010087,2.199581e+06,0.004691,1.446002e+06,0.052142,5.324126e+06,0.004859,7.505155e+03,...,0.004306,1.243972e+04,0.021002,1.375474e+06,0.012102,2.249621e+06,0.004079,1.284862e+06,0.004163,1.875621e+03
1701912024,0.005208,3.785331e+06,0.009313,2.565518e+06,0.005312,2.847608e+03,0.055225,3.495819e+06,0.004952,3.089686e+06,...,0.003260,9.878573e+03,0.020117,1.535452e+06,0.012409,2.310013e+02,0.004184,2.399579e+06,0.003959,1.735775e+06


In [4]:
nodes_df.to_csv(os.path.join(DATA_PATH, "raw", "nodes.csv"))

In [4]:
edges_file = os.path.join(DATA_PATH, "raw", "edges.csv")
edges_df = pd.read_csv(edges_file)

edges_df["value"] = (edges_df["value"] - edges_df["value"].mean()) / edges_df["value"].std()
edges_df["value"] = (edges_df["value"] - edges_df["value"].min()) / (edges_df["value"].max() - edges_df["value"].min())
edges_df

,from,to,timestamp,value
0,frontend,adservice,1701825744,0.105048
1,frontend,adservice,1701825804,0.090534
2,frontend,adservice,1701825864,0.087769
3,frontend,adservice,1701825924,0.102974
4,frontend,adservice,1701825984,0.091916
...,...,...,...,...
20155,frontend,shippingservice,1701911904,0.038700
20156,frontend,shippingservice,1701911964,0.032481
20157,frontend,shippingservice,1701912024,0.035936
20158,frontend,shippingservice,1701912084,0.039391


In [5]:
edges_df = edges_df[edges_df["timestamp"].isin(nodes_df.index)]
edges_df

,from,to,timestamp,value
0,frontend,adservice,1701825744,0.105048
1,frontend,adservice,1701825804,0.090534
2,frontend,adservice,1701825864,0.087769
3,frontend,adservice,1701825924,0.102974
4,frontend,adservice,1701825984,0.091916
...,...,...,...,...
20155,frontend,shippingservice,1701911904,0.038700
20156,frontend,shippingservice,1701911964,0.032481
20157,frontend,shippingservice,1701912024,0.035936
20158,frontend,shippingservice,1701912084,0.039391


In [6]:
edges_df.to_csv(os.path.join(DATA_PATH, "raw", "edges.csv"), index=False)

## RAW data to NumPy files

### Nodes

In [7]:
nodes_df = nodes_df.sort_values("timestamp")
edges_df = edges_df.sort_values("timestamp")

In [8]:
window = 60
steps = 3
train_split = 0.8

In [9]:
X_nodes = []
y_nodes = []

for i in tqdm(range(0, len(nodes_df) - window, steps)):
    node_window = nodes_df.iloc[i : i + window]
    node_horizont = nodes_df.iloc[i + window]
    
    x = []
    y = []
    for pod in pod_names:
        x.append(node_window[[f"{pod}_cpu", f"{pod}_ram"]].to_numpy())
        y.append(node_horizont[[f"{pod}_cpu", f"{pod}_ram"]].to_numpy())
    X_nodes.append(np.array(x).swapaxes(0, 1))
    y_nodes.append(np.array(y))

X_nodes = np.array(X_nodes)
y_nodes = np.array(y_nodes)

100%|██████████| 460/460 [00:02<00:00, 218.66it/s]


In [10]:
X_nodes[:, :, :, 1] = X_nodes[:, :, :, 1] / 1e6
y_nodes[:, :, 1] = y_nodes[:, :, 1] / 1e6

In [11]:
split_point = int(len(X_nodes) * train_split)

np.savez("../data/train_node_features.npz", 
         X=X_nodes[:split_point], 
         y=y_nodes[:split_point])

np.savez("../data/test_node_features.npz", 
         X=X_nodes[split_point:], 
         y=y_nodes[split_point:])

### Edges

In [12]:
edge_timed = edges_df.groupby("timestamp")

In [13]:
X_edges = []
y_edges = []
graph = []

for k in tqdm(range(0, len(nodes_df) - window, steps)):
    nodes = nodes_df.iloc[k : k + window + 1]
    if len(graph) == window + 1:
        nodes = nodes.iloc[-steps:]
        graph = graph[steps:]

    for t in nodes.index:
        edges_info = edge_timed.get_group(t)
        A = np.zeros((len(pod_names), len(pod_names)))
        for i in range(len(pod_names)):
            for j in range(len(pod_names)):
                from_pod = pod_names[i]
                to_pod = pod_names[j]
                
                query = edges_info.query("`from` == @from_pod and `to` == @to_pod")
                if len(query) > 0:
                    A[i, j] = query["value"].item()
        graph.append(A)
    X_edges.append(graph[:window])
    y_edges.append(graph[window])

X_edges = np.array(X_edges)
y_edges = np.array(y_edges)

100%|██████████| 460/460 [02:48<00:00,  2.73it/s]


In [14]:
np.savez("../data/train_edge_features.npz", 
         X=X_edges[:split_point], 
         y=y_edges[:split_point])

np.savez("../data/test_edge_features.npz", 
         X=X_edges[split_point:], 
         y=y_edges[split_point:])